In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, ConcatDataset, SubsetRandomSampler
import pandas as pd
import numpy as np
import random

import os
import cv2
import time
from sklearn.model_selection import train_test_split

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet \
--save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \
'https://docs.google.com/uc?export=download&id=1vwucTNj66q93cRYjkzKVNxHd5k3Xpm5f' -O- | sed -rn 's/.*confirm=([0-9A-\
!Za-z_]+).*/\1\n/p')&id=1OfvUx1jCva-TR97_GsguAQpd_0rP4HOx" -O train_100Hz.zip && rm -rf /tmp/cookies.txt
!unzip train_100Hz.zip

sed: -e expression #1, char 38: Invalid range end

--2023-05-22 02:18:39--  https://docs.google.com/uc?export=download&confirm=&id=1OfvUx1jCva-TR97_GsguAQpd_0rP4HOx

Resolving docs.google.com (docs.google.com)... 74.125.141.139, 74.125.141.113, 74.125.141.101, ...

Connecting to docs.google.com (docs.google.com)|74.125.141.139|:443... connected.

HTTP request sent, awaiting response... 303 See Other

Location: https://doc-0k-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tcijk7ubv1ilrlsp75edmrab2otqkjtt/1684721850000/02832111001387922702/*/1OfvUx1jCva-TR97_GsguAQpd_0rP4HOx?e=download&uuid=e93c302c-d343-4648-bc61-b1a1ff41a868 [following]


--2023-05-22 02:18:42--  https://doc-0k-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tcijk7ubv1ilrlsp75edmrab2otqkjtt/1684721850000/02832111001387922702/*/1OfvUx1jCva-TR97_GsguAQpd_0rP4HOx?e=download&uuid=e93c302c-d343-4648-bc61-b1a1ff41a868

Resolving doc-0k-64-docs.googleusercontent.com (

In [ ]:
def delete_ds_store(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file == ".DS_Store":
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print("Deleted:", file_path)

delete_ds_store("./train_100Hz/")

Deleted: ./train_100Hz/.DS_Store


In [ ]:
#sheet = pd.read_csv('/content/sample_1.csv')
#data_label = sheet.iloc[:,:].values
#print(data_label.shape[0])

def cut_data(data_label, size, overlap):
    data = np.zeros((1,size))
    label = np.zeros((1,size))
    end = size
    start = 0 
    length = data_label.shape[0]
    while end < length:
      data = np.append(data, data_label[start:end,0].reshape((1,size)), axis =0)
      label = np.append(label, data_label[start:end,1].reshape((1,size)), axis =0)
      start = end - overlap 
      end = start + size
    else:
      data = np.append(data, data_label[length - size:length,0].reshape((1,size)), axis =0)
      label = np.append(label,data_label[length - size:length,1].reshape((1,size)), axis =0)
      
    return data[1:], label[1:] 

In [ ]:
def calculate_signal_ratio(arr):
    count_ones = np.count_nonzero(arr == 1)
    ratio_ones = count_ones / arr.size
    return ratio_ones

In [ ]:
folder_path = './train_100Hz/'
files = os.listdir(folder_path)
size = 700
train_x = np.zeros((1,size))
train_y = np.zeros((1,size))

test_x = np.zeros((1,size))
test_y = np.zeros((1,size))

for i, file in enumerate(files):  
    print(file)
    sample = pd.read_csv(os.path.join(folder_path, file))
    data_label = sample.iloc[:,:].values
    #print(calculate_signal_ratio(data_label[:,1]))
    #print(data_label.shape)
    signal_ratio = calculate_signal_ratio(data_label[:,1])
    test_list = ['1_1679771170.914631_label.csv', '1_1679332756.399197_label.csv', '18_1682404281.14869_RawData_LabelData.csv', \
                 '22_1682491900.592947_RawData_LabelData.csv']
    if file in test_list:
        t_data_add, t_label_add = cut_data(data_label, size, 50)
        test_x = np.concatenate((test_x, t_data_add), axis =0)
        test_y = np.concatenate((test_y, t_label_add), axis =0)
    else:
        if signal_ratio > 0.4:
            data_label = np.concatenate((data_label, data_label, data_label, data_label, data_label, data_label), axis=0)
            #print(calculate_signal_ratio(data_label[:,1]))
        if data_label.shape[0] > size: 
            data_add, label_add = cut_data(data_label, size, 50)
            train_x = np.concatenate((train_x, data_add), axis =0)
            train_y = np.concatenate((train_y, label_add), axis =0)
            if calculate_signal_ratio(label_add)>0.7:
                 train_x = np.concatenate((train_x, data_add), axis =0)
                 train_y = np.concatenate((train_y, label_add), axis =0)

In [13]:
train_x = train_x[1:]
train_x = train_x.astype(np.float32)
train_y = train_y[1:] 
train_y = train_y.astype(np.int64)

test_x = test_x[1:]
test_x = test_x.astype(np.float32)
test_y = test_y[1:] 
test_y = test_y.astype(np.int64)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(6761, 440)

(6761, 440)

(414, 440)

(414, 440)


In [14]:
print(f"Ratio of ones: {calculate_signal_ratio(train_y)}")
print(f"Ratio of ones: {calculate_signal_ratio(test_y)}")

Ratio of ones: 0.4885005580132041

Ratio of ones: 0.5172101449275363


In [ ]:
import pywt

# 定義連續小波轉換（CWT）函數
def cwt_transform(data):
    wavelet = 'morl'  # 使用Morlet小波
    scales = np.arange(1, 129)  # 定義尺度範圍

    cwt_coeffs = []
    for sample in data:
        coeffs, _ = pywt.cwt(sample, scales, wavelet)
        cwt_coeffs.append(coeffs)

    cwt_coeffs = np.array(cwt_coeffs)
    return cwt_coeffs


# 執行CWT轉換
train_x_cwt = cwt_transform(train_x)
test_x_cwt = cwt_transform(test_x)

# 確保train_x_cwt的維度與train_x相同
print(train_x_cwt.shape)  # (number, 128, 700)
print(test_x_cwt.shape)

In [ ]:
def average_pooling(cwt_result, scale_factor):
    #(number, 32, 4, 700)
    return np.mean(cwt_result.reshape(cwt_result.shape[0], -1, scale_factor, cwt_result.shape[2]), axis=2)


scale_factor = 4
train_x_cwt = average_pooling(train_x_cwt, scale_factor)
test_x_cwt = average_pooling(test_x_cwt, scale_factor)

print(train_x_cwt.shape)  # (number, 32, 700)
print(test_x_cwt.shape)

In [ ]:
# Define the amplitude scaling range
scale_range = (0.5, 1.5)

# Define the amplitude scaling transformation
data_transform = transforms.Lambda(lambda x: x * random.uniform(scale_range[0], scale_range[1]))

In [15]:
class EPG_Dataset(Dataset):
    def __init__(self, x, x_cwt, y, transform=None):
        self.x = x
        self.x_cwt = x_cwt
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        X = self.x[index]
        CWT = self.x_cwt[index]
        Y = self.y[index]

        if self.transform is not None:
            X = self.transform(X)

        return X, CWT, Y

In [16]:
valid_num = int(train_x.shape[0]*0.2)
train_num = train_x.shape[0] - valid_num

batch_size = 500
whole_set = EPG_Dataset(train_x, train_x_cwt, train_y)
test_set = EPG_Dataset(test_x, test_x_cwt, test_y)
train_set, valid_set = torch.utils.data.random_split(whole_set,[train_num,valid_num])

whole_loader = DataLoader(whole_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True)

print('Size of testing data:', len(test_set))
print('Size of training data:', len(train_set))
print('Size of testing data:', len(valid_set))

Size of training data: 5409

Size of testing data: 1352


In [ ]:
class EPGTransformerCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(EPGTransformerCNN, self).__init__()
        #self.cnn = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.cnn1d = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1), #[16,size/2]
            nn.MaxPool1d(2, 2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
        
            nn.Conv1d(16, 32, 3, 1, 1),  #[32, size/4]
            nn.MaxPool1d(2, 2),
            nn.BatchNorm1d(32),
            nn.ReLU(),   
        )     
        self.cnn2d = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1), #[8,16,size/2]
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(8),
            nn.ReLU(),
        
            nn.Conv2d(8, 16, 3, 1, 1),  #[16,8,size/4]
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(16),
            nn.ReLU(), 
        )     
        self.fc1 = nn.Sequential(
            nn.Linear(8*size, 1000),
            nn.Dropout(0.15),
            nn.ReLU(),
            nn.Linear(1000, 150),
            nn.Dropout(0.15),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(4*8*size, 5000),
            nn.Dropout(0.15),
            nn.ReLU(),
            nn.Linear(5000, 1000),
            nn.Dropout(0.15),
            nn.ReLU(),
            nn.Linear(1000, 150),
            nn.Dropout(0.15),
        )
        self.transformer = nn.Transformer(d_model=hidden_size, nhead=4, num_encoder_layers=2)
        self.out = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, x_cwt):
        x = x.unsqueeze(1)  # (batch_size, 1, size)
        out_x = self.cnn1d(x)
        out_x = out_x.view(out_x.size()[0], -1)  # (batch_size, 16*size)
        out_x = self.fc1(out_x) 
        
        x_cwt = x_cwt.unsqueeze(1)
        out_cwt = self.cnn2d(x_cwt)
        out_cwt = out_cwt.view(out_cwt.size()[0], -1) 
        out_cwt = self.fc2(out_cwt) 
        
        out = torch.cat((out_x, out_cwt), dim=1)
        
        out = self.transformer(out, out)
        out = self.out(out)
        out = self.sigmoid(out)
        return out

In [ ]:
def testing(model):
    model.eval()
    test_acc, test_loss = 0.0,0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            test_pred = model(data[0].cuda(), data[1].cuda())
            batch_loss = criterion(test_pred, data[2].cuda().float())
            test_acc += np.sum(torch.round(test_pred.cpu()).data.numpy() == data[2].numpy())
            #test_loss += batch_loss.item()
            #print(torch.round(test_pred.cpu()).data.numpy())
            #print(data[1])
            #print("Test Acc:{}, Test Loss:{}".format(test_acc/(test_set.__len__()*size), test_loss/(test_set.__len__()*size)))
            
    return test_acc/(test_set.__len__()*size)

[[0. 0. 0. ... 0. 0. 0.]

 [0. 0. 0. ... 0. 0. 0.]

 [1. 1. 1. ... 1. 1. 1.]

 ...

 [0. 0. 0. ... 0. 0. 0.]

 [1. 1. 1. ... 1. 1. 1.]

 [0. 0. 0. ... 0. 0. 0.]]

tensor([[0, 0, 0,  ..., 0, 0, 0],

        [1, 1, 1,  ..., 1, 1, 1],

        [0, 0, 0,  ..., 1, 1, 1],

        ...,

        [0, 0, 0,  ..., 0, 0, 0],

        [1, 1, 1,  ..., 1, 1, 1],

        [1, 1, 1,  ..., 1, 1, 1]])

[[0. 0. 0. ... 0. 0. 0.]

 [1. 1. 1. ... 1. 1. 1.]

 [1. 1. 1. ... 1. 1. 1.]

 ...

 [0. 0. 0. ... 0. 0. 0.]

 [0. 0. 0. ... 0. 0. 0.]

 [1. 1. 1. ... 1. 1. 1.]]

tensor([[1, 1, 1,  ..., 1, 1, 1],

        [1, 1, 1,  ..., 1, 1, 1],

        [1, 1, 1,  ..., 1, 1, 1],

        ...,

        [0, 0, 0,  ..., 0, 0, 0],

        [0, 0, 0,  ..., 0, 0, 0],

        [1, 1, 1,  ..., 1, 1, 1]])

[[1. 1. 1. ... 1. 1. 1.]

 [1. 1. 1. ... 1. 1. 1.]

 [0. 0. 0. ... 0. 0. 0.]

 ...

 [1. 1. 1. ... 1. 1. 1.]

 [0. 0. 0. ... 0. 0. 0.]

 [0. 0. 0. ... 0. 0. 0.]]

tensor([[0, 0, 0,  ..., 0, 0, 0],

        [1, 1, 1,  ..., 1,

In [ ]:
input_size = size
hidden_size = 300
output_size = size

model = EPGTransformerCNN(input_size, hidden_size, output_size).cuda()

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)
num_epoch = 85
train_epochs = []
test_epochs = []

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0

    model.train()
    for i, data in enumerate(whole_loader):
        optimizer.zero_grad()
        train_pred = model(data[0].cuda(), data[1].cuda())
        batch_loss = criterion(train_pred, data[2].cuda().float())
        batch_loss.backward()
        optimizer.step()
        train_acc += np.sum(torch.round(train_pred.cpu()).data.numpy().reshape(-1,size) == data[2].numpy())
        train_loss += batch_loss.item()
        #print(torch.round(train_pred.cpu()).data.numpy().reshape(-1,size))
        #print(data[1].numpy())
        
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/(whole_set.__len__()*size), train_loss/(whole_set.__len__()*size)))
    
    if (epoch+1) % 5 == 0: 
        train_epochs.append(train_acc/(whole_set.__len__()*size))
        a = testing(model)
        print(a)
        test_epochs.append(a)

print(train_epochs)
print(test_epochs)

[001/025] 0.73 sec(s) Train Acc: 0.498615 Loss: 0.000004

[002/025] 0.62 sec(s) Train Acc: 0.498994 Loss: 0.000003

[003/025] 0.64 sec(s) Train Acc: 0.499418 Loss: 0.000003

[004/025] 0.62 sec(s) Train Acc: 0.500186 Loss: 0.000003

[005/025] 0.62 sec(s) Train Acc: 0.500333 Loss: 0.000003

[006/025] 0.63 sec(s) Train Acc: 0.501426 Loss: 0.000003

[007/025] 0.63 sec(s) Train Acc: 0.500986 Loss: 0.000003

[008/025] 0.62 sec(s) Train Acc: 0.501150 Loss: 0.000003

[009/025] 0.62 sec(s) Train Acc: 0.502365 Loss: 0.000003

[010/025] 0.62 sec(s) Train Acc: 0.507947 Loss: 0.000003

[011/025] 0.62 sec(s) Train Acc: 0.524899 Loss: 0.000003

[012/025] 0.64 sec(s) Train Acc: 0.567226 Loss: 0.000003

[013/025] 0.72 sec(s) Train Acc: 0.615871 Loss: 0.000003

[014/025] 0.74 sec(s) Train Acc: 0.664300 Loss: 0.000003

[015/025] 0.73 sec(s) Train Acc: 0.720675 Loss: 0.000003

[016/025] 0.63 sec(s) Train Acc: 0.763101 Loss: 0.000003

[017/025] 0.63 sec(s) Train Acc: 0.798421 Loss: 0.000003

[018/025] 0.64

In [ ]:
torch.save(model.state_dict(), './model_ep34.pt')

In [ ]:
model.eval()
test_acc, test_loss = 0.0,0

with torch.no_grad():
        for i, data in enumerate(test_loader):
            test_pred = model(data[0].cuda(), data[1].cuda())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def draw_pic(index):
    a = data[0].cpu().numpy()[index, :]
    b = torch.round(test_pred.cpu()).data.numpy()[index, :]
    x = np.arange(size)

    fig, ax = plt.subplots()

    # 绘制折线图
    ax.plot(x, a, color='gray', label='data')

    # 绘制 a 中 b 为 1 的点，颜色为红色，大小为 5
    ax.scatter(x[b == 1], a[b == 1], c='red', label='signal', s=5)

    # 绘制 a 中 b 为 0 的点，颜色为蓝色，大小为 5
    ax.scatter(x[b == 0], a[b == 0], c='blue', label='noise', s=5)

    ax.legend()
    ax.set_xlabel('Index')
    ax.set_ylabel('Value')

    plt.show()
    filename = f'plot_prediction_{index}.png'
    fig.savefig(filename)


for i in range(20):
    draw_pic(i)

In [ ]:
#lr=0.00001
#num_epoch = 15, [015/015] 0.55 sec(s) Train Acc: 0.614801 Loss: 0.000003 Test Acc:0.6178011643528885
#num_epoch = 17, [017/017] 0.53 sec(s) Train Acc: 0.592808 Loss: 0.000003 Test Acc:0.5648612750885478
#num_epoch = 20, [020/020] 0.54 sec(s) Train Acc: 0.800360 Loss: 0.000002 Test Acc:0.4484848484848485
#num_epoch = 20, [020/020] 0.56 sec(s) Train Acc: 0.769532 Loss: 0.000003 Test Acc:0.8133448184468592  ***
#num_epoch = 20 [020/020] 0.58 sec(s) Train Acc: 0.677996 Loss: 0.000003 Test Acc:0.48254545454545456
#num_epoch = 25, [025/025] 0.57 sec(s) Train Acc: 0.818442 Loss: 0.000002 Test Acc:0.29112383679312814

#lr=0.000005
#num_epoch = 25, [025/025] 0.59 sec(s) Train Acc: 0.696756 Loss: 0.000003 Test Acc:0.5743006993006993
#num_epoch = 30, [030/030] 0.54 sec(s) Train Acc: 0.655785 Loss: 0.000003 Test Acc:0.522711038961039
#num_epoch = 33  [033/033] 0.55 sec(s) Train Acc: 0.763456 Loss: 0.000003 Test Acc:0.7260200429491768
#num_epoch = 34  [034/034] 0.65 sec(s) Train Acc: 0.802218 Loss: 0.000002 Test Acc:0.7817303469477382
#num_epoch = 34  [034/034] 0.55 sec(s) Train Acc: 0.795996 Loss: 0.000003 Test Acc:0.8938148609381487 ***
#num_epoch = 34  [034/034] 0.54 sec(s) Train Acc: 0.801585 Loss: 0.000002 Test Acc:0.7659365393061045
#num_epoch = 35, [035/035] 0.56 sec(s) Train Acc: 0.804331 Loss: 0.000002 Test Acc:0.4920320855614973

#[0.5928 0.6148 0.6557 0.6780 0.6967 0.7634 0.7695 0.7960 0.8004 0.8022 0.8043 0.81844]
#[0.5649 0.6178 0.5227 0.4825 0.5743 0.7260 0.8133 0.8938 0.4485 0.7817 0.4920 0.2911] 

#(5733, 440)
#(198, 440)

#Transform/Test num =198
#lr=0.000005
#[040/040] 0.48 sec(s) Train Acc: 0.825898 Loss: 0.000002 Test Acc:0.5975665748393021
#[060/060] 0.48 sec(s) Train Acc: 0.827733 Loss: 0.000002 Test Acc:0.6296946740128558
#[080/080] 0.46 sec(s) Train Acc: 0.857112 Loss: 0.000002 Test Acc:0.6735078053259872
#Transform 0.7-1.3/Test num =198
#lr=0.000005
#[100/100] 0.47 sec(s) Train Acc: 0.854072 Loss: 0.000002 Test Acc:0.6317837465564738
#[120/120] 0.48 sec(s) Train Acc: 0.864163 Loss: 0.000002 Test Acc:0.659159779614325
 
#larger model + transform(0.7,1.3)
#lr=0.000005
#[080/080] 0.49 sec(s) Train Acc: 0.884652 Loss: 0.000002 Test Acc:0.654912764003673
#[100/100] 0.48 sec(s) Train Acc: 0.921284 Loss: 0.000001 Test Acc:0.690805785123967
#[120/120] 0.50 sec(s) Train Acc: 0.941723 Loss: 0.000001 Test Acc:0.7061179981634527
#[140/140] 0.49 sec(s) Train Acc: 0.943499 Loss: 0.000001 Test Acc:0.7431473829201102 + 以下都加dropout(0.1)
#[160/160] 0.48 sec(s) Train Acc: 0.951038 Loss: 0.000001 Test Acc:0.6757575757575758
#[180/180] 0.49 sec(s) Train Acc: 0.960227 Loss: 0.000001 Test Acc:0.7108356290174472
#+dropout(0.15)
#[140/140] 0.49 sec(s) Train Acc: 0.923211 Loss: 0.000001 Test Acc:0.7408057851239669
#[140/140] 0.48 sec(s) Train Acc: 0.919488 Loss: 0.000001 Test Acc:0.6913452708907254
#+dropout(0.15)+ transform(0.5,1.5)
#[150/150] 0.49 sec(s) Train Acc: 0.914206 Loss: 0.000001 Test Acc:0.6730716253443526
#+dropout(0.2)
#[140/140] 0.49 sec(s) Train Acc: 0.944264 Loss: 0.000001 Test Acc:0.7093204775022957

In [ ]:
#+CWT & size= 700
#epoch = 150
#Test Acc: [0.49467857 0.49645238 0.50172619 0.55271429 0.64540476 0.70619048 
#0.72659524 0.75847619 0.73310714 0.72279762 0.71645238 0.75432143
#0.7649881  0.75603571 0.76640476 0.76761905 0.76778571 0.77469048
#0.75641667 0.75457143 0.79108333 0.75794048 0.75528571 0.71125
#0.75777381 0.69659524 0.70514286 0.70814286 0.75230952 0.7524881]